# Overview
Getting proteomes for Turbellaria and Cestoda for further analyses.

___

Running in Obdulio server

___

**Note**: this notebook might me temporal, or might be conserved in order to keep the valuable info of how this was got in a separate notebook

# Creating relevant folders and getting raw files

In [1]:
from ftplib import FTP # importing libraries
import os, subprocess, glob

def create_dir(dir): # define auxiliary function
    if not os.path.exists(dir):
        os.mkdir(dir)

cestode_genome_species = ['Dibothriocephalus latus', 'Echinococcus canadensis', 'Echinococcus granulosus',
                          'Echinococcus multilocularis', 'Echinococcus oligarthrus', 'Hydatigera taeniaeformis',
                          'Hymenolepis diminuta', 'Hymenolepis microstoma', 'Hymenolepis nana',
                          'Mesocestoides corti', 'Schistocephalus solidus', 'Spirometra erinaceieuropaei',
                          'Taenia asiatica', 'Taenia multiceps', 'Taenia saginata', 'Taenia solium']

monogenean_genome_species = ['Gyrodactylus salaris', 'Protopolystoma xenopodis'] # list of monogeneans
turbellarian_genome_species = ['Macrostomum lignano', 'Schmidtea mediterranea'] # put list of turbellarians

# create dir to allocate
create_dir('../orthologous_groups/data/set_cestoda_monogenea_turbellaria')
create_dir('../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set')
create_dir('../orthologous_groups/data/set_cestoda_monogenea_turbellaria/renamed_set')

In [2]:
# connecting to server
current_dir = os.getcwd() # get current folder
os.chdir('../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set') # move to folder to download

with FTP('ftp.ebi.ac.uk') as ftp:
    ftp.login('anonymous', '') # login to server
    # move to location of WBPS15
    ftp.cwd('pub/databases/wormbase/parasite/releases/WBPS15/species')
    #print(ftp.nlst())
    for species_dir in ftp.nlst():
        if ' '.join(species_dir.split('_')).capitalize() in cestode_genome_species+monogenean_genome_species+turbellarian_genome_species:
            print(species_dir) # print species just to know
            ftp.cwd(species_dir)
            for assembly in ftp.nlst():
                ftp.cwd(assembly)
                proteome_file = [file for file in ftp.nlst() if 'protein' in file][0] # get files
                if not os.path.exists(proteome_file):
                    with open(proteome_file, 'wb') as file: # download file
                        ftp.retrbinary(f"RETR {proteome_file}", file.write)
                ftp.cwd('../')
            ftp.cwd('/pub/databases/wormbase/parasite/releases/WBPS15/species')

os.chdir(current_dir) # get to original folder

dibothriocephalus_latus
echinococcus_canadensis
echinococcus_granulosus
echinococcus_multilocularis
echinococcus_oligarthrus
gyrodactylus_salaris
hydatigera_taeniaeformis
hymenolepis_diminuta
hymenolepis_microstoma
hymenolepis_nana
macrostomum_lignano
mesocestoides_corti
protopolystoma_xenopodis
schistocephalus_solidus
schmidtea_mediterranea
spirometra_erinaceieuropaei
taenia_asiatica
taenia_multiceps
taenia_saginata
taenia_solium


# Renaming sequences for further analyses

## Get already used sequence codes

In [3]:
import pandas as pd
correspondence_codes_table = pd.read_csv('../orthologous_groups/data/trematode_set/code_correspondence_isoform_aware.tsv', sep = '\t')
employed_codes = list(set([label.split('.')[0] for label in correspondence_codes_table['new_label'].to_list()]))

In [4]:
employed_codes

['ABQ',
 'ABD',
 'ABR',
 'ABI',
 'ABF',
 'ABM',
 'ABO',
 'ABP',
 'ABN',
 'ABS',
 'ABC',
 'ABJ',
 'ABK',
 'ABL',
 'ABE',
 'ABG',
 'ABH']

## Creating a three-letter code independent of that already created for trematodes
Just using a four letter code obtained from the publication *Compositional analysis of flatworm genomes shows a strong codon usage bias across all classes* (https://www.frontiersin.org/articles/10.3389/fgene.2019.00771/full) (of our group) and PlanMine (that uses a similar code).

- *E. granulosus*: Egra
- E. multilocularis - Emu
- Hymenolepis diminuta - Hdim
- Mesocestoides corti - Mcor
- Schistocephalus solidus - Ssol
- Opistorchis viverrini - Oviv
- Clonorchis sinensis - Csin
- Fasciola hepatica - Fhep
etc...


In [5]:
species2code = {species: species.split(' ')[0][0]+species.split(' ')[1][0:3].lower() for species in cestode_genome_species+monogenean_genome_species+turbellarian_genome_species} # create a dictionary species-code

Now taking into account same scheme to get codes for trematodes and rename OGs to get a coherent naming frame!

In [29]:
# now for trematodes
oldtremcode2new_dict = {} # create dictionary for old code2new

for fasta_file in glob.glob('../orthologous_groups/data/trematode_set/renamed_set/*'):
    # get the species name
    trematodes_species = ' '.join(fasta_file.rpartition('/')[2].split('.')[0].split('_')).capitalize()
    # generating new code for that trematode
    new_trematode_code = trematodes_species.split(' ')[0][0]+trematodes_species.split(' ')[1][0:3].lower()
    # get current code
    current_code = list(set([record.id.split('.')[0] for record in SeqIO.parse(fasta_file, 'fasta')]))[0]
    if current_code == 'ABG':
        # update <oldtremcode2new_dict>
        oldtremcode2new_dict.update({current_code: 'Fbus'})
        # update <species2code>
        species2code.update({'Fasciolopsis buski': 'Fbus'})
    else:
        # update <oldtremcode2new_dict>
        oldtremcode2new_dict.update({current_code: new_trematode_code})
        # update <species2code>
        species2code.update({trematodes_species: new_trematode_code})

In [30]:
oldtremcode2new_dict

{'ABP': 'Smar',
 'ABE': 'Fgig',
 'ABM': 'Shae',
 'ABI': 'Oviv',
 'ABC': 'Csin',
 'ABG': 'Fbus',
 'ABQ': 'Smat',
 'ABS': 'Treg',
 'ABO': 'Sman',
 'ABH': 'Ofel',
 'ABD': 'Ecap',
 'ABJ': 'Pwes',
 'ABK': 'Sbov',
 'ABN': 'Sjap',
 'ABR': 'Srod',
 'ABL': 'Scur',
 'ABF': 'Fhep'}

In [31]:
species2code

{'Dibothriocephalus latus': 'Dlat',
 'Echinococcus canadensis': 'Ecan',
 'Echinococcus granulosus': 'Egra',
 'Echinococcus multilocularis': 'Emul',
 'Echinococcus oligarthrus': 'Eoli',
 'Hydatigera taeniaeformis': 'Htae',
 'Hymenolepis diminuta': 'Hdim',
 'Hymenolepis microstoma': 'Hmic',
 'Hymenolepis nana': 'Hnan',
 'Mesocestoides corti': 'Mcor',
 'Schistocephalus solidus': 'Ssol',
 'Spirometra erinaceieuropaei': 'Seri',
 'Taenia asiatica': 'Tasi',
 'Taenia multiceps': 'Tmul',
 'Taenia saginata': 'Tsag',
 'Taenia solium': 'Tsol',
 'Gyrodactylus salaris': 'Gsal',
 'Protopolystoma xenopodis': 'Pxen',
 'Macrostomum lignano': 'Mlig',
 'Schmidtea mediterranea': 'Smed',
 'Schistosoma margrebowiei': 'Smar',
 'Fasciola gigantica': 'Fgig',
 'Schistosoma haematobium': 'Shae',
 'Opisthorchis viverrini': 'Oviv',
 'Clonorchis sinensis': 'Csin',
 'Schistosoma mattheei': 'Smat',
 'Trichobilharzia regenti': 'Treg',
 'Schistosoma mansoni': 'Sman',
 'Opisthorchis felineus': 'Ofel',
 'Echinostoma capro

In [22]:
# create function to perform operation over OGs with parallelization
def rename_trematode_og(fasta_file, outdir):
    fasta_filename = fasta_file.rpartition('/')[2] # get fasta name
    fasta_records = [record for record in SeqIO.parse(fasta_file, 'fasta')] # get records
    # iterate over sequences on the FASTA and replace, for each record ID, to the replace old code with new
    for record in fasta_records:
        species_code = record.id.split('.')[0] # get species code
        seq_number = record.id.split('.')[1] # get the number of the sequence
        record.id = oldtremcode2new_dict[species_code] + '.' + seq_number # rename record id
        record.name = ''
        record.description = ''
    # save fasta if not already present
    fasta_outname = outdir + '/' + fasta_filename
    if not os.path.exists(fasta_outname):
        with open(fasta_outname, 'w') as handle_fasta:
            SeqIO.write(fasta_records, handle_fasta, 'fasta')

In [32]:
# create folders to allocate renamed OGs
create_dir('../og_trimal_analyses/data/og_full_set/ogs_purged_by_species_and_length_with_more_than_one_seq_newtremcode')

# operate in concurrent way for all relevant files
import concurrent.futures
import multiprocessing

avail_threads = multiprocessing.cpu_count()
with concurrent.futures.ThreadPoolExecutor(max_workers = (avail_threads - 1)) as executor:
        # operate over all OGs, and then see what happends
        for og_fasta in glob.glob('../og_trimal_analyses/data/og_full_set/ogs_purged_by_species_and_length_with_more_than_one_seq/*.fa'):
            outdir = '../og_trimal_analyses/data/og_full_set/ogs_purged_by_species_and_length_with_more_than_one_seq_newtremcode'
            args = (og_fasta, outdir)
            executor.map(rename_trematode_og(*args), chunksize = 10)

## Renaming

In [7]:
import gzip # import library
from Bio import SeqIO

# create vector to allocate info on this seqs
name_correspondence_rows = []
for fasta_gzip in glob.glob('../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set/*.fa'):
    print(fasta_gzip)
    # get species name
    splitted_line = fasta_gzip.rpartition('/')[2].split('.')[0].split('_')
    species_name = ''.join(splitted_line[0].capitalize() + ' ' + splitted_line[1])
    # get species code
    species_code = species2code[species_name]
    #fasta_records =  # get the FASTA records
    # renaming with a loop
    # saving sequences
    species_filename = fasta_gzip.rpartition('/')[2].rpartition('.')[0]
    output_fasta = '../orthologous_groups/data/set_cestoda_monogenea_turbellaria/renamed_set/{0}.renamed.fa'.format(species_filename)
    if not os.path.exists(output_fasta):        
        fasta_records = [record for record in SeqIO.parse(fasta_gzip, 'fasta')]
        for record_data in enumerate(fasta_records, start = 1):
            index_num, record = record_data # unpack
            # rename the sequence
            #print(record)
            seq_name = species_code + '.' + str(index_num)
            # append to vector of seq names
            name_correspondence_rows.append(pd.DataFrame.from_dict({'seq_full_id': [record.description],  'code': [seq_name]}))
            record.id = seq_name
            record.name = seq_name
            record.description = seq_name
            with open(output_fasta, 'a') as handle_fasta:
                SeqIO.write(record, handle_fasta, 'fasta')

# create table
name_correspondence_table = pd.concat(name_correspondence_rows)

../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set/taenia_saginata.PRJNA71493.WBPS15.protein.fa
../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set/hydatigera_taeniaeformis.PRJEB534.WBPS15.protein.fa
../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set/echinococcus_canadensis.PRJEB8992.WBPS15.protein.fa
../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set/mesocestoides_corti.PRJEB510.WBPS15.protein.fa
../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set/macrostomum_lignano.PRJNA284736.WBPS15.protein.fa
../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set/hymenolepis_diminuta.PRJEB507.WBPS15.protein.fa
../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set/hymenolepis_diminuta.PRJEB30942.WBPS15.protein.fa
../orthologous_groups/data/set_cestoda_monogenea_turbellaria/original_set/taenia_asiatica.PRJEB532.WBPS15.protein.fa
../orthologous_groups

ValueError: No objects to concatenate

In [ ]:
name_correspondence_table.head()

In [ ]:
# save correspondence table
name_correspondence_table.to_csv('../orthologous_groups/data/set_cestoda_monogenea_turbellaria/cestoda_monogenea_turbellaria_index_code.tsv', 
                                 sep = '\t', index = False, header = True)